<a href="https://colab.research.google.com/github/endzix22/machine-learning-bootcamp/blob/main/UM3_ekstarcja_cech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ekstarcja cech- przygotowanie danych do UM

W większości metod głębokiego uczenia (KNN), same wydobywają one np. cechy zaszyte w obrazach. 
W pzoostałych metodach ta ekstracja przebiega często ręcznie

Co zrobić, aby wydobyć te cechy z danych?
np. utworzyc jakis stosunek z danych juz zawartych w tabeli...
wynika to z tego, czy my dobrze sie orientujemy w jakies dziedzinie, stanowi to wazny etap w modelu 


Przyklad uzycia:
masz kolumne ze wzrostem, next podziel na 3 kategorie w zaleznosci od tej kolumny np. wysoki, niski, sredni, next zastosuj to kodowanie 0,1 co robilas dla kolumn category

In [1]:
import numpy as np 
import pandas as pd 
import sklearn 


Zaladowanie danych

In [3]:

def fetch_financial_data(company='AMZN'):
    """
    funckja domyslnie pobierze notowania firmy AMAZON
    """
    import pandas_datareader.data as web
    return web.DataReader(name=company, data_source='stooq')
    #'stooq', portal finansowy z notowaniami

df_raw = fetch_financial_data()
df_raw.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-04-01,3282.990,3316.54,3246.39,3271.20,2854475
2022-03-31,3328.895,3329.90,3259.07,3259.95,2998289
2022-03-30,3370.190,3379.01,3310.00,3326.02,2808363
2022-03-29,3407.680,3416.63,3357.37,3386.30,3307680
2022-03-28,3299.500,3380.75,3298.00,3379.81,2992691


Utworzenie kopii danych

In [6]:
df=df_raw.copy()
df=df[:5]
#wytnijmy sobie na potrzeby zadania tylko 5 pierwszych wierszy
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2022-04-01 to 2022-03-28
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


Generowanie nowych zmiennych, 
czyt nowych kolumn do juz istniejacych wyzej

In [8]:
#uwaga zostaly utworzone kolumny, gdzie z indeksu 
#czyt. daty zostaly wyodrebnione wartosci do kolumn
df['day']=df.index.day
df['month']=df.index.month
df['year']=df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2022-04-01,3282.990,3316.54,3246.39,3271.20,2854475,1,4,2022
2022-03-31,3328.895,3329.90,3259.07,3259.95,2998289,31,3,2022
2022-03-30,3370.190,3379.01,3310.00,3326.02,2808363,30,3,2022
2022-03-29,3407.680,3416.63,3357.37,3386.30,3307680,29,3,2022
2022-03-28,3299.500,3380.75,3298.00,3379.81,2992691,28,3,2022


Dyskretyzacja zmiennej ciaglej

In [9]:
#wygenerowanie przykaldowych danych:
df=pd.DataFrame(data={'height':[175.,178.5,185.,191.,184.5,183.,168.]})

metoda cut 
bins=ile podzialow, 
wtedy zostaly dopasowane przedzialy

In [10]:
df['height_cat']=pd.cut(x=df.height,bins=3)
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


Mozna podac konkretne przedzialy (samemu):

In [12]:
df['height_cat']=pd.cut(x=df.height,bins=(160,175,180,195))
df
#zostaly tutaj utworzone 2 przedzialy:
#1: 160-175 
#2.180-195

,height,height_cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


Tutaj dokonujemy podzialu na przedzialy w zaleznosci do kolmny wzrostu, ale nie chcemy aby w kolumnie wynikowej wysiwetlal sie przedzial tylko np. napis 'small'/'medium'/'high'

---



In [14]:
df['height_cat']=pd.cut(x=df.height,bins=(160,175,180,195),labels=['small','medium','high'])
df

,height,height_cat
0,175.0,small
1,178.5,medium
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


Teraz mozemy uzyc get_dummies, w celu przygotowania danych do naszego modelu

In [16]:
pd.get_dummies(df,drop_first=True,prefix='height')
#jedna kolumna ze zbioru: small/medium/high, zostanie usunieta 
#bo kodowanie n-1

,height,height_medium,height_high
0,175.0,0,0
1,178.5,1,0
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,183.0,0,1
6,168.0,0,0


Ekstracja cech 

In [18]:
#generowanie dancyh do przykladu 
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


In [19]:
#chcemy zalozmy wydobyc ile poszczegolny wiersz zawiera jezykow

In [21]:
df['lang_number']=df['lang'].apply(len)
df

,lang,lang_number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


In [22]:
#mozna takze budowac flage o to czy w poszczegolnych wierszach
#wystepuje jezyk pl

In [23]:
df['PL_flag']=df['lang'].apply(lambda x:1 if 'PL' in x else 0)
df

,lang,lang_number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


In [24]:
#wygenerowanie inncyh danych do dalszej ekstarcji:

In [25]:
df=pd.DataFrame(data={'website':['wp.pl','onet.pl','google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [33]:
#podzielic string wzgledem znaku '.'
df.website.str.split('.',expand=True)

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [37]:
new=df.website.str.split('.',expand=True)
df['portal']=new[0]
df['extension']=new[1]
df
#taki myk zeby przypisac nowe nazwy do kolumn

,website,portal,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
